In [1]:
from __future__ import absolute_import #?
from __future__ import division #?
from __future__ import print_function #?

import pandas as pd
import numpy as np
import os

import time
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()
import pickle
import sys

import warnings
warnings.filterwarnings(action='once')

import torch #?
import torch.utils.data

# BERT
#package_dir = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT" # doesnt contain GPT2ClassificationHeadModel apparently
#sys.path.append(package_dir)
sys.path.insert(0, "../input/gpt2-pytorch/pytorch-pretrained-bert-master/pytorch-pretrained-BERT-master")
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from pytorch_pretrained_bert import BertConfig

# LSTM
from keras.preprocessing import sequence, text #
from keras.models import load_model

# FASTAI
from torch import nn # FASTAI
from torch.nn import functional as F # FASTAI

# GPT2
#sys.path.insert(0, "../input/gpt2-pytorch/pytorch-pretrained-bert-master/pytorch-pretrained-BERT-master")
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2ClassificationHeadModel
from pytorch_pretrained_bert import OpenAIAdam

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:5506: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/.keras/keras.json' mode='r' encoding='UTF-8'>
  _config = json.load(open(_config_path))
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 

In [2]:
INPUT_PATH = '../input'
#JIGSAW_PATH = '../input/jigsaw-unintended-bias-in-toxicity-classification'

UTILS = ['jigsaw-unintended-bias-in-toxicity-classification',
         'ppbert',
         'bert-pretrained-models',
         'gpt2-pytorch',
         'gpt2-models',         
        ]

MODEL_PATHS = [p for p in os.listdir('../input') if p not in UTILS]
MODEL_PATHS

['d-fastai-n5-e4-s4',
 'd-lstm-m-n2-e6-s456-2',
 'd-bert-pytorch-15-soft-targets-w-t15e-10',
 'pickled-crawl300d2m-for-kernel-competitions',
 'd-gpt2-pytorch-m-1515-st-w-1',
 'pickled-glove840b300d-for-10sec-loading',
 'd-bert-pytorch-15-soft-targets-w-t15e-1',
 'd-gpt2-pytorch-m2f-1515-1',
 'd-bert-pytorch-15-soft-targets-w-t15e-4',
 'd-gpt2-pytorch-m2-1515-taw3-lrscos-1',
 'd-bert-pytorch-15-soft-targets-w-t15e-2',
 'd-bert-pytorch-15-soft-targets-w-t15e-3']

In [3]:
MODEL_PATHS_BERT = [os.path.join(INPUT_PATH, p) for p in os.listdir(INPUT_PATH) if p not in UTILS and p.split('-')[1] == 'bert']
MODEL_PATHS_BERT

['../input/d-bert-pytorch-15-soft-targets-w-t15e-10',
 '../input/d-bert-pytorch-15-soft-targets-w-t15e-1',
 '../input/d-bert-pytorch-15-soft-targets-w-t15e-4',
 '../input/d-bert-pytorch-15-soft-targets-w-t15e-2',
 '../input/d-bert-pytorch-15-soft-targets-w-t15e-3']

In [4]:
MODEL_PATHS_LSTM = [os.path.join(INPUT_PATH, p) for p in os.listdir(INPUT_PATH) if p not in UTILS and p.split('-')[1] == 'lstm']
MODEL_PATHS_LSTM

['../input/d-lstm-m-n2-e6-s456-2']

In [5]:
MODEL_PATHS_FASTAI = [os.path.join(INPUT_PATH, p) for p in os.listdir(INPUT_PATH) if p not in UTILS and p.split('-')[1] == 'fastai']
MODEL_PATHS_FASTAI

['../input/d-fastai-n5-e4-s4']

In [6]:
MODEL_PATHS_GPT2 = [os.path.join(INPUT_PATH, p) for p in os.listdir(INPUT_PATH) if p not in UTILS and p.split('-')[1] == 'gpt2']
MODEL_PATHS_GPT2

['../input/d-gpt2-pytorch-m-1515-st-w-1',
 '../input/d-gpt2-pytorch-m2f-1515-1',
 '../input/d-gpt2-pytorch-m2-1515-taw3-lrscos-1']

## TEST DATAFRAME

In [7]:
test_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
test_df.head()

,id,comment_text
0,7097320,[ Integrity means that you pay your debts.]\n\...
1,7097321,This is malfeasance by the Administrator and t...
2,7097322,@Rmiller101 - Spoken like a true elitist. But ...
3,7097323,"Paul: Thank you for your kind words. I do, in..."
4,7097324,Sorry you missed high school. Eisenhower sent ...


## FUNCTIONS

### BERT FUNCTIONS

In [8]:
BERT_PRETRAINED_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/' # contains tokenizer
BERT_PYTORCH_PATH = '../input/d-bert-pytorch-15-soft-targets-w-t15e-3' # contains pretrained model converted to pytorch (better to save/load only one)

MAX_SEQUENCE_LENGTH_BERT = 220
NUM_LABELS_BERT = 6
BATCH_SIZE_BERT = 32
#BATCH_SIZE_BERT = 768 # a bit faster but riskier because of RAM usage

DEVICE = torch.device('cuda')

In [9]:
def preprocess_bert():
    start = time.time()
    
    ## grab tokenizer from pretrained BERT
    tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_MODEL_PATH, cache_dir=None, do_lower_case=True)
    
    # Converting the lines to BERT format
    # Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
    def convert_lines_bert(example, max_seq_length, tokenizer):
        max_seq_length -=2
        all_tokens = []
        longer = 0
        for text in tqdm(example):
            tokens_a = tokenizer.tokenize(text)
            if len(tokens_a)>max_seq_length:
                tokens_a = tokens_a[:max_seq_length]
                longer += 1
            one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
            all_tokens.append(one_token)
        #print(longer)
        return np.array(all_tokens)
    
    ## test read & conversion to BERT format
    #test_df['comment_text'] = test_df['comment_text'].astype(str) # we want test_df intact
    X_test_bert = convert_lines_bert(test_df["comment_text"].astype(str).fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH_BERT, tokenizer)

    print(f'{time.time() - start:.2f} seconds')
    return X_test_bert    

In [10]:
def bert_predict(model_path, X_test, batch_size=BATCH_SIZE_BERT):
    start = time.time()
    
    # loading pytorch model (pytorch_model.bin) and config (bert_config.json)
    model = BertForSequenceClassification.from_pretrained(BERT_PYTORCH_PATH, cache_dir=None, num_labels=NUM_LABELS_BERT)

    # loading model state 
    model.load_state_dict(torch.load(os.path.join(model_path, 'bert_pytorch.bin'))['state_dict'])

    # setting model in eval mode
    model.to(DEVICE)
    for param in model.parameters():
        param.requires_grad=False
    model.eval()
    
    # predicting
    test_preds = np.zeros((len(X_test)))
    test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    for i,(x_batch,) in enumerate(tqdm(test_loader)):
        pred = model(x_batch.to(DEVICE), attention_mask=(x_batch>0).to(DEVICE), labels=None)
        test_preds[i*batch_size:(i+1)*batch_size]=pred[:,0].detach().cpu().squeeze().numpy()
    test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()
    
    print(f'{time.time() - start:.2f} seconds')
    return test_pred    

### LSTM FUNCTIONS

In [11]:
CHARS_TO_ISOLATE_LSTM = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'
CHARS_TO_REMOVE_LSTM = '_`\t\n₹' # applied by tokenizer

MAX_LEN_LSTM = 220
BATCH_SIZE_LSTM = 2048

In [12]:
def clean_text_lstm(x, puncts):
    x = str(x)
    for punct in puncts:
        if punct in x: 
            x = x.replace(punct, f' {punct} ')
    return x

In [13]:
EMBEDDING_FILES = [
    '../input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl',
    '../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
]
N_EMBS_LSTM = 328444 # we need to build an emb matrix with the same shape as the old one

In [14]:
def load_embeddings_new(path):
    with open(path, 'rb') as f:
        emb_arr = pickle.load(f)
        #return dict(get_coefs(*line.strip().split(' ')) for line in f)
    return emb_arr


def build_matrix_new(word_index, path):
    embedding_index = load_embeddings_new(path)
    #embedding_matrix = np.zeros((len(word_index) + 1, 300)) # we need to build a matrix with the same shape so we can swap the weights
    # otherwise 'ValueError: Layer weight shape (328444, 600) not compatible with provided weight shape (75216, 600)'
    embedding_matrix = np.zeros((N_EMBS_LSTM, 300))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

In [15]:
def preprocess_lstm():
    start = time.time()

    X_test_lstm = test_df['comment_text'].astype(str)
    X_test_lstm = X_test_lstm.progress_apply(lambda x: clean_text_lstm(x, CHARS_TO_ISOLATE_LSTM))

    # loading tokenizer
    with open(os.path.join(MODEL_PATHS_LSTM[0],'tokenizer.pickle'), 'rb') as handle:
        tokenizer = pickle.load(handle)

    X_test_lstm = tokenizer.texts_to_sequences(X_test_lstm)
    X_test_lstm = sequence.pad_sequences(X_test_lstm, maxlen=MAX_LEN_LSTM)

    print(f'{time.time() - start:.2f} seconds')    
    return X_test_lstm    

In [16]:
def preprocess_lstm_emb():
    start = time.time()

    X_test_lstm = test_df['comment_text'].astype(str)
    X_test_lstm = X_test_lstm.progress_apply(lambda x: clean_text_lstm(x, CHARS_TO_ISOLATE_LSTM))

    # creating tokenizer
    tokenizer = text.Tokenizer(filters=CHARS_TO_REMOVE_LSTM)
    tokenizer.fit_on_texts(list(X_test_lstm))    

    X_test_lstm = tokenizer.texts_to_sequences(X_test_lstm)
    X_test_lstm = sequence.pad_sequences(X_test_lstm, maxlen=MAX_LEN_LSTM)
    
    emb_matrix_new = np.concatenate(
    [build_matrix_new(tokenizer.word_index, f) for f in EMBEDDING_FILES], axis=-1)

    print(f'{time.time() - start:.2f} seconds')    
    return X_test_lstm, emb_matrix_new

In [17]:
def lstm_predict_emb(model_path, X_test, embedding_matrix_new, batch_size=BATCH_SIZE_LSTM, checkpoints=[], weights=[]):
    start = time.time()
    
    all_test_pred = []
    for checkpoint in checkpoints:
    
        # LOADING MODEL
        model = load_model(os.path.join(model_path, checkpoint))
        model.layers[1].set_weights([embedding_matrix_new]) #

        # PREDICTING
        test_pred = model.predict(X_test, batch_size=BATCH_SIZE_LSTM)[0].flatten()
        all_test_pred.append(test_pred)
        
    test_pred_lstm = np.average(all_test_pred, weights=weights, axis=0)
    
    print(f'{time.time() - start:.2f} seconds')
    return test_pred_lstm   

In [18]:
def lstm_predict(model_path, X_test, batch_size=BATCH_SIZE_LSTM, checkpoints=[], weights=[]):
    start = time.time()
    
    all_test_pred = []
    for checkpoint in checkpoints:
    
        # LOADING MODEL
        model = load_model(os.path.join(model_path, checkpoint))

        # PREDICTING
        test_pred = model.predict(X_test, batch_size=BATCH_SIZE_LSTM)[0].flatten()
        all_test_pred.append(test_pred)
        
    test_pred_lstm = np.average(all_test_pred, weights=weights, axis=0)
    
    print(f'{time.time() - start:.2f} seconds')
    return test_pred_lstm    

### FASTAI FUNCTIONS

In [19]:
MAX_LEN_FASTAI = 300
MAX_FEATURES_FASTAI = 410047 # it wont be used if we create new emb matrix
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
NUM_AUX_TARGETS_FASTAI = 6 #['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']
BATCH_SIZE_FASTAI = 512

In [20]:
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix, num_aux_targets):
        super(NeuralNet, self).__init__()
        embed_size = embedding_matrix.shape[1]
        
        #self.embedding = nn.Embedding(MAX_FEATURES_FASTAI, embed_size) # better with embedding_matrix directly
        self.embedding = nn.Embedding(embedding_matrix.shape[0]-1, embed_size) #
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)
        
    def forward(self, x, lengths=None):
        h_embedding = self.embedding(x.long())
        h_embedding = self.embedding_dropout(h_embedding)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        # global average pooling
        avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)
        
        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_conc))
        h_conc_linear2  = F.relu(self.linear2(h_conc))
        
        hidden = h_conc + h_conc_linear1 + h_conc_linear2
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out
    
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

In [21]:
CRAWL_EMBEDDING_PATH = '../input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl'
GLOVE_EMBEDDING_PATH = '../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr


def build_matrix(word_index, path, max_features):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((max_features + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        if i <= max_features:
            try:
                embedding_matrix[i] = embedding_index[word]
            except KeyError:
                try:
                    embedding_matrix[i] = embedding_index[word.lower()]
                except KeyError:
                    try:
                        embedding_matrix[i] = embedding_index[word.title()]
                    except KeyError:
                        unknown_words.append(word)
    return embedding_matrix, unknown_words

In [22]:
SYMBOLS_TO_ISOLATE_FASTAI = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
SYMBOLS_TO_DELETE_FASTAI = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'

isolate_dict = {ord(c):f' {c} ' for c in SYMBOLS_TO_ISOLATE_FASTAI}
remove_dict = {ord(c):f'' for c in SYMBOLS_TO_DELETE_FASTAI}


def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

def handle_contractions(x):
    from nltk.tokenize.treebank import TreebankWordTokenizer
    tokenizer = TreebankWordTokenizer()
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def preprocess(x):
    x = handle_punctuation(x)
    x = handle_contractions(x)
    x = fix_quote(x)
    return x

In [23]:
def preprocess_fastai():
    start = time.time()
    
    with open(os.path.join(MODEL_PATHS_FASTAI[0], 'tokenizer.pickle'), 'rb') as handle:
        tokenizer = pickle.load(handle)

    X_test_fastai = test_df['comment_text'].progress_apply(lambda x: preprocess(x))
    X_test_fastai = tokenizer.texts_to_sequences(X_test_fastai)
    X_test_fastai = sequence.pad_sequences(X_test_fastai, maxlen=MAX_LEN_FASTAI)

    print(f'{time.time() - start:.2f} seconds')    

    return X_test_fastai

In [24]:
def preprocess_fastai_emb():
    start = time.time()

    X_test_fastai = test_df['comment_text'].progress_apply(lambda x: preprocess(x))
    
    # creating new tokenizer
    tokenizer = text.Tokenizer(num_words = MAX_FEATURES_FASTAI, filters='', lower=False)
    tokenizer.fit_on_texts(list(X_test_fastai))

    X_test_fastai = tokenizer.texts_to_sequences(X_test_fastai)
    X_test_fastai = sequence.pad_sequences(X_test_fastai, maxlen=MAX_LEN_FASTAI)
    
    max_features_fastai = min(MAX_FEATURES_FASTAI, len(tokenizer.word_index)) # its gonna be len(tokenizer.word_index) if we only fit on test    
    crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH, max_features_fastai)
    glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH, max_features_fastai)
    emb_matrix_new = np.concatenate([crawl_matrix, glove_matrix], axis=-1)

    print(f'{time.time() - start:.2f} seconds')    

    return X_test_fastai, emb_matrix_new

In [25]:
def fastai_predict_emb(model_path, X_test, embedding_matrix_new, batch_size=BATCH_SIZE_FASTAI, checkpoints=[], weights=[]):
    start = time.time()
    
    all_test_pred = []
    for checkpoint in checkpoints:
    
        # LOADING MODEL        
        model = NeuralNet(embedding_matrix_new, NUM_AUX_TARGETS_FASTAI)
        state = torch.load(os.path.join(model_path, checkpoint)) # state_dict saved
        state['embedding.weight'] = torch.tensor(embedding_matrix_new)
        model.load_state_dict(state)

        # PREDICTING
        model.cuda()
        model.eval()

        test_preds = np.zeros((len(X_test)))
        test_tensor = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
        test_loader = torch.utils.data.DataLoader(test_tensor, batch_size=batch_size, shuffle=False)
        for i, (x_batch,) in enumerate(tqdm(test_loader)):
            pred = model(x_batch.cuda())    
            test_preds[i * batch_size:(i + 1) * batch_size] = pred[:, 0].detach().cpu().squeeze().numpy()
        test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy()        
        all_test_pred.append(test_pred)
        
    test_pred_fastai = np.average(all_test_pred, weights=weights, axis=0)
    
    print(f'{time.time() - start:.2f} seconds')
    return test_pred_fastai    

In [26]:
def fastai_predict(model_path, X_test, batch_size=BATCH_SIZE_FASTAI, checkpoints=[], weights=[]):
    start = time.time()
    
    all_test_pred = []
    for checkpoint in checkpoints:
    
        # LOADING MODEL
        state = torch.load(os.path.join(model_path, checkpoint)) # state_dict saved
        embedding_matrix = state['embedding.weight']
        model = NeuralNet(embedding_matrix, NUM_AUX_TARGETS_FASTAI)
        model.load_state_dict(state)

        # PREDICTING
        model.cuda()
        model.eval()

        test_preds = np.zeros((len(X_test)))
        test_tensor = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
        test_loader = torch.utils.data.DataLoader(test_tensor, batch_size=batch_size, shuffle=False)
        for i, (x_batch,) in enumerate(tqdm(test_loader)):
            pred = model(x_batch.cuda())    
            test_preds[i * batch_size:(i + 1) * batch_size] = pred[:, 0].detach().cpu().squeeze().numpy()
        test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy()        
        all_test_pred.append(test_pred)
        
    test_pred_fastai = np.average(all_test_pred, weights=weights, axis=0)
    
    print(f'{time.time() - start:.2f} seconds')
    return test_pred_fastai    

### GPT2 FUNCTIONS

In [27]:
GPT2_PRETRAINED_MODEL_PATH = '../input/gpt2-models' # contains tokenizer AND pytorch converted model

MAX_SEQUENCE_LENGTH_GPT2 = 220
NUM_LABELS_GPT2 = 6
BATCH_SIZE_GPT2 = 32

DEVICE = torch.device('cuda')

In [28]:
def preprocess_gpt2():
    start = time.time()
    
    # grab tokenizer from pretrained
    tokenizer = GPT2Tokenizer.from_pretrained(GPT2_PRETRAINED_MODEL_PATH)
    
    # converting the lines to GPT2 format
    def convert_lines_gpt2(example, max_seq_length, tokenizer):
        all_tokens = []
        for text in tqdm(example):
            tokens_a = tokenizer.tokenize(text)    
            if len(tokens_a) > MAX_SEQUENCE_LENGTH_GPT2:
                tokens_a = tokens_a[-MAX_SEQUENCE_LENGTH_GPT2:]
            one_token = tokenizer.convert_tokens_to_ids(tokens_a) + [0]*(MAX_SEQUENCE_LENGTH_GPT2 - len(tokens_a))
            all_tokens.append(one_token)
        return np.array(all_tokens)
    
    # test read & conversion to GPT2 format
    #test_df['comment_text'] = test_df['comment_text'].astype(str) 
    X_test_gpt2 = convert_lines_gpt2(test_df["comment_text"].astype(str).fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH_GPT2, tokenizer)

    print(f'{time.time() - start:.2f} seconds')

    return X_test_gpt2 

In [29]:
def gpt2_predict(model_path, X_test, batch_size=BATCH_SIZE_GPT2):
    start = time.time()  
    
    # loading gpt2 pytorch model (looks for pytorch_model.bin AND config.json)
    #model = GPT2ClassificationHeadModel.from_pretrained(model_path, clf_dropout=0.4, n_class=NUM_LABEL_GPT2) # WORKS even when pytorch_model.bin only contains state_dict
    model = GPT2ClassificationHeadModel.from_pretrained(GPT2_PRETRAINED_MODEL_PATH, clf_dropout=0.4, n_class=NUM_LABELS_GPT2) # ? SHOULD WORK

    # loading model state 
    model.load_state_dict(torch.load(os.path.join(model_path, 'pytorch_model.bin'))['state_dict'])

    # setting model in eval mode
    model.to(DEVICE)
    for param in model.parameters():
        param.requires_grad=False
    model.eval()
    
    # predicting
    test_preds = np.zeros((len(X_test)))
    test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    for i,(x_batch,) in enumerate(tqdm(test_loader)):
        pred = model(x_batch.to(DEVICE))
        test_preds[i*batch_size:(i+1)*batch_size]=pred[:,0].detach().cpu().squeeze().numpy()
    test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()
    
    print(f'{time.time() - start:.2f} seconds')
                          
    return test_pred    

## ENSEMBLE

### LSTM

### ~ 10 s

In [30]:
#X_test_lstm = preprocess_lstm()
X_test_lstm, emb_matrix_new = preprocess_lstm_emb()

#6.50 seconds
#29.99 seconds


32.32 seconds


### ~ 2 m 10 s *(~25s per ckpt)*

In [31]:
MODEL_PATHS_LSTM

['../input/d-lstm-m-n2-e6-s456-2']

In [32]:
predictions_lstm = []
predictions_lstm += [
                        #lstm_predict('../input/d-lstm-m-n2-e6-s456-2',
                        lstm_predict_emb('../input/d-lstm-m-n2-e6-s456-2', #                                                                                                                                                                                    
                                     X_test_lstm,
                                     emb_matrix_new, #
                                     checkpoints=['LSTM_1_4.h5',
                                                  'LSTM_1_5.h5',
                                                  'LSTM_1_6.h5',
                                                  'LSTM_2_4.h5',
                                                  'LSTM_2_5.h5',
                                                  'LSTM_2_6.h5',
                                                 ],
                                     weights=[1,1,1,1,1,1],
                                     )
                     ]
prediction_lstm = np.average(predictions_lstm, weights=None, axis=0)

#129.14 seconds

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/keras/engine/saving.py:251: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an

150.80 seconds


In [33]:
from numba import cuda
cuda.select_device(0)
cuda.close()

### FASTAI

### ~ 35 s

In [34]:
#X_test_fastai = preprocess_fastai()
X_test_fastai, emb_matrix_new = preprocess_fastai_emb() #

#34.10 seconds

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)



60.53 seconds


### ~ 1 m 30 s *(~20s per ckpt)*

In [35]:
MODEL_PATHS_FASTAI

['../input/d-fastai-n5-e4-s4']

In [36]:
predictions_fastai = []
predictions_fastai += [
                       #fastai_predict('../input/d-fastai-n5-e4-s4',
                       fastai_predict_emb('../input/d-fastai-n5-e4-s4', #                                                                        
                                     X_test_fastai,
                                     emb_matrix_new, #
                                     checkpoints=['state_1_4.bin',
                                                  'state_2_4.bin',
                                                  'state_3_4.bin',
                                                  'state_4_4.bin',
                                                  'state_5_4.bin',
                                                 ],
                                     weights=[1,1,1,1,1],
                                     )
                     ]
prediction_fastai = np.average(predictions_fastai, weights=None, axis=0)

#85.71 seconds
#107.72 seconds


92.48 seconds


### GPT2

### ~ 45 s

In [37]:
X_test_gpt2 = preprocess_gpt2()

#42.42 seconds

../input/gpt2-pytorch/pytorch-pretrained-bert-master/pytorch-pretrained-BERT-master/pytorch_pretrained_bert/tokenization_gpt2.py:146: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2-models/vocab.json' mode='r' encoding='UTF-8'>
  self.encoder = json.load(open(vocab_file))
../input/gpt2-pytorch/pytorch-pretrained-bert-master/pytorch-pretrained-BERT-master/pytorch_pretrained_bert/tokenization_gpt2.py:151: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2-models/merges.txt' mode='r' encoding='utf-8'>
  bpe_data = open(merges_file, encoding='utf-8').read().split('\n')[1:-1]



40.44 seconds


### ~ 13 m 30 s *(~14m per model)*

In [38]:
predictions_gpt2 = [gpt2_predict(path, X_test_gpt2) for path in MODEL_PATHS_GPT2]
prediction_gpt2 = np.average(predictions_gpt2, weights=None, axis=0)

#789.48 seconds
#793.64 seconds #cuda 'closed'


791.91 seconds


### BERT

### ~ 2 m

In [39]:
X_test_bert = preprocess_bert()

#116.52 seconds


119.45 seconds


### ~ 12 m *(~12m per model)*

In [40]:
predictions_bert = [bert_predict(path, X_test_bert) for path in MODEL_PATHS_BERT]
prediction_bert = np.average(predictions_bert, weights=None, axis=0)

#719.34 seconds


722.42 seconds



721.90 seconds


## ENSEMBLE

In [41]:
predictions = []
predictions.append(prediction_bert)
predictions.append(prediction_lstm)
predictions.append(prediction_fastai)
predictions.append(prediction_gpt2)

In [42]:
WEIGHT_BERT = 4   # 4 models
WEIGHT_LSTM = .75   # 1 model 
WEIGHT_FASTAI = .75   # 1 model
WEIGHT_GPT2 = 2   # 1 model

weights = [WEIGHT_BERT, WEIGHT_LSTM, WEIGHT_FASTAI, WEIGHT_GPT2]
weights

[4, 0.75, 0.75, 2]

In [43]:
prediction = np.average(predictions,
                        weights=weights,
                        axis=0,
                       )

In [44]:
submission = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': prediction,
})

submission.head(10)

,id,prediction
0,7097320,0.008471
1,7097321,0.065478
2,7097322,0.263842
3,7097323,0.058740
4,7097324,0.010455
5,7097325,0.008007
6,7097326,0.801430
7,7097327,0.487713
8,7097328,0.005724
9,7097329,0.047942


In [45]:
submission.to_csv('submission.csv', index=False)